In [ ]:
#@title Initialize and prepare Whisper environment {display-mode: "form"}

%pip install -q git+https://github.com/openai/whisper.git
%pip install -q ffmpeg-python
%pip install -q python-docx
%pip install -q srt
!git clone -b master https://github.com/bhnatt/whisper-am.git

### https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
#@title input parameters {display-mode: "form"}
import os
import sys
if os.path.isdir ('/content/whisper-am') :
    sys.path.append ('/content/whisper-am')

from whisper_am import WhisperAM, mountDrive

data_dir = 'data' #@param {type:"string"}
if data_dir [-1] != '/' : data_dir = data_dir + '/'

ds = mountDrive ()
google_drive = '/content/drive/MyDrive/' if ds else './'
data_path   = google_drive + data_dir
model_name = 'tiny.en' #@param ["medium.en", "large", "small.en', "tiny.en"]

am = WhisperAM (model_name, data_path)
if am.checkCuda () != 'cuda' :
    print ('need to change colab runtime to GPU...')
    if ds : exit ()

In [ ]:
#%%time
#@title run Whisper {display-mode: "form"}
import time

print ('Start :', time.strftime('%X %x %Z'))
am.run ()
print ('End  :', time.strftime('%X %x %Z'))